Model Architecture:

1. Embedding Layer:
   - The input text data is converted into dense vectors using an embedding layer. Each word is represented as a fixed-size vector in a high-dimensional space.

2. Bidirectional LSTM Layers:
   - The embedding vectors are passed through Bidirectional LSTM (Long Short-Term Memory) layers. LSTM is a type of recurrent neural network (RNN) that can capture long-term dependencies in sequential data.
   - The Bidirectional LSTM layers process the input sequences in both forward and backward directions, capturing information from past and future contexts.

3. Dropout Layer:
   - A dropout layer is added to prevent overfitting. Dropout randomly drops a fraction of input units to zero during training, which helps in regularization and improves generalization.

4. Dense Layers:
   - The output from the LSTM layers is fed into fully connected dense layers. These layers perform non-linear transformations on the input data, allowing the model to learn complex patterns.

5. Output Layer:
   - The final dense layer with a softmax activation function generates probabilities for the next word in the sequence. The word with the highest probability is selected as the predicted next word.

6. Compilation:
   - The model is compiled using the categorical cross-entropy loss function and the Adam optimizer. Categorical cross-entropy is commonly used for multi-class classification tasks, while Adam is an efficient optimizer for training neural networks.

Training:
   - The model is trained on the input sequences and corresponding labels (one-hot encoded). During training, the model adjusts its parameters (weights and biases) to minimize the loss function, i.e., the difference between predicted and actual outputs.

Model Summary:
   - The model summary provides a detailed overview of the layers, their output shapes, and the number of trainable parameters.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Read the text data file
data = open('poem.txt', encoding="utf8").read()

# Generate corpus by splitting the text into lines
corpus = data.lower().split("\n")

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)

# Generate input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

# Model architecture
model = Sequential()
model.add(Embedding(total_words+1, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
print(model.summary())

# Training the model
history = model.fit(predictors, label, epochs=50, verbose=1)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           65600     
                                                                 
 bidirectional_1 (Bidirecti  (None, 13, 300)           301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 13, 300)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 656)               66256     
                                                                 
Total params: 593456 (2.26 MB)
Trainable params: 593456 (2.26 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
# Save the model
model.save("NLP.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# To load the saved model
"""
from tensorflow.keras.models import load_model
loaded_model = load_model("NLP.h5")
"""

In [ ]:
def poem():

  # Generate text
  seed_text = "The world"
  next_words = 100
  output_text = ""

  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted = np.argmax(model.predict(token_list), axis=-1)
      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              break
      seed_text += " " + output_word

  return seed_text

1/1 [==============================] - 0s 34ms/step
The world to love to strangers of was me known of was me loud of do of do do do the mournful bleak bound rose erase do the stone rose erase of day do do this in the stone rose own can to do the stone hours erase of was do me now from me the stone rose erase beside the stone hours erase of was do do the mournful sea hours hours rose erase do the echoes hours erase do the echoes hours erase of was me loud rose of was me loud beside the stone rose rose erase of day
